# Création d'une base de données enrichie avec les cractéristiques physico-chimiques des bières.
L'idée est la suivante : on a récolté toutes les bières du site www.beerwulf.com. On peut donc ensuite faire le lien avec toutes les données du dataset préexistant. Cela permettra de passer de environ 1000 données à 100 000 données ?

In [146]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [152]:
df = pd.read_csv('data_beerwulf')
df.head()

,Unnamed: 0,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,umami,price,note
0,Brugse Zot blonde,Brugse Zot blonde,Bière Blonde,Belgique,Brouwerij De Halve Maan,33.0,6.0,23.0,60.0,40.0,20.0,40.0,0.0,2.69,3.71
1,Brouwerij 't IJ Natte Dubbel,Brouwerij 't IJ Natte Dubbel,Bière Double,Pays-Bas,Brouwerij 't IJ,33.0,6.5,NaN,60.0,60.0,0.0,40.0,0.0,2.65,3.44
2,Bruut Gajes Tripel,Bruut Gajes Tripel,Bière Triple,Pays-Bas,Bruut Bier,33.0,8.0,38.0,60.0,60.0,40.0,60.0,0.0,2.99,3.76
3,Baxbier Koud Vuur,Baxbier Koud Vuur,Bière Brune,Pays-Bas,Bax Bier Brouwerij,33.0,6.2,38.0,80.0,40.0,20.0,60.0,0.0,3.09,3.37
4,Bacchus Framboos,Bacchus Framboos,Bière Fruitée,Belgique,Brouwerij Van Honsebrouck,38.0,5.0,NaN,80.0,60.0,60.0,20.0,0.0,3.49,3.60


La 1ère colonne ne nous intéresse pas, on la supprime.

In [153]:
del df['Unnamed: 0']
df.describe()

,bottle,abv,ibu,intensite,longueur,acidite,amertume,umami,price,note
count,977.000000,977.000000,555.000000,919.000000,919.000000,919.000000,919.000000,919.000000,827.000000,906.000000
mean,33.210850,6.245241,35.619820,59.151251,52.557127,13.949946,44.787813,3.199129,2.877110,3.473532
std,3.913847,2.167882,20.013929,16.427631,16.577893,17.033918,20.079733,9.554029,0.747449,0.285906
min,25.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.190000,1.850000
25%,33.000000,5.000000,22.000000,60.000000,40.000000,0.000000,40.000000,0.000000,2.490000,3.340000
50%,33.000000,6.000000,32.000000,60.000000,60.000000,20.000000,40.000000,0.000000,2.790000,3.500000
75%,33.000000,7.500000,44.500000,60.000000,60.000000,20.000000,60.000000,0.000000,2.990000,3.660000
max,65.000000,16.500000,220.000000,100.000000,100.000000,80.000000,100.000000,80.000000,10.990000,4.450000


Ces statistiques descriptives nous permettent de visualiser plusieurs problèmes : 
- La valeur minimum de `abv` est 0, ce qui veut dire que nous avons scrapé des bières sans alcool. Néanmoins, la valeur maximale (16.5°) semble cohérente.
- La valeur maximale de l'`ibu` est louche.
- L'`umami` des bières collectées est égale à 0 pour au moins 75% des données scrapées.
- Le `prix` semble honnête, mais la valeur maximale semble étrange. De plus, le prix n'est pas uniformisé dans le sens où il correspond à un volume (`bottle`) précis. Il faudra donc ramener le tout au prix de la pinte de bière. 
- La `note` de satisfation de la bière est bien comprise entre 0 et 5 mais son écart-type est très faible. On aura donc du mal à distinguer les très bonnes bières et les bonnes bières.



### On nettoie le dataframe afin de ne garder que les bières sans alcool

In [162]:
bieres_sans_alcool = df[df['abv'].isin([0,np.nan])] 
bieres_sans_alcool.head(7)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,umami,price,note
20,"Budels Malt 0,0%",Lager,Pays-Bas,Budelse Brouwerij,30.0,0.0,NaN,40.0,60.0,20.0,20.0,0.0,1.79,2.11
22,Heineken 0.0,Lager,Pays-Bas,Heineken Brewery,30.0,0.0,16.0,40.0,40.0,40.0,40.0,20.0,1.49,2.30
66,Palm 0.0,Pale Ale,Belgique,Brouwerij Palm,25.0,0.0,NaN,60.0,60.0,20.0,40.0,0.0,1.89,2.06
75,Braxzz Porter,Bière Porter & Stout,Pays-Bas,NaN,33.0,0.0,50.0,60.0,80.0,0.0,40.0,0.0,2.49,2.51
89,Affligem Blond 0.0%,Bière Blonde,Belgique,Affligem,30.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.59,2.84
215,Lowlander 0% Wit,Bière Blanche,Pays-Bas,Lowlander,33.0,0.0,NaN,40.0,40.0,40.0,0.0,0.0,2.49,3.16
289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


D'ailleurs, on remarque que les bières sans alcool n'ont pas de "bonne" note. On les supprime du dataframe.

In [163]:
df = df.drop([df[df['abv'].isin([0,np.nan])].index[i] for i in range(len(df[df['abv'].isin([0,np.nan])]))])
df = df.reset_index(drop = True)

### On lève les doutes sur l'`ibu` des bières

In [159]:
df.sort_values('ibu', ascending=False).head(5)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,umami,price,note
379,Brewdog Jack Hammer,IPA,Royaume-Uni,Brewdog,33.0,7.2,220.0,100.0,60.0,20.0,100.0,0.0,2.99,3.92
881,Viven Imperial IPA,IPA,Belgique,Brouwerij Van Viven,33.0,8.0,124.0,80.0,40.0,20.0,80.0,0.0,2.99,3.72
968,Kees Export Porter 1750,Bière Porter & Stout,Pays-Bas,Brouwerij Kees,33.0,10.5,110.0,100.0,60.0,20.0,100.0,40.0,3.69,3.97
699,Dochter Van De Korenaar Extase,IPA,Belgique,NaN,33.0,7.0,110.0,80.0,60.0,0.0,80.0,0.0,3.19,3.71
622,De Molen Hel & Verdoemenis,Bière Porter & Stout,Pays-Bas,Brouwerij De Molen,33.0,10.0,102.0,80.0,60.0,20.0,80.0,40.0,4.99,4.06


Après vérification sur le site des brasseries des bières ayant la plus grande concentration d'IBU, ces données sont vraie. On ne modifie pas le dataframe.

# Inspection de l'`umami` des bières

## Manque de données, donc on tej cette colonne => à modifier dans le fichier 'Enrichissement de la base de données'.

### On lève les doutes sur le `prix` des bières

In [160]:
df.sort_values('price', ascending=False).head(5)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,umami,price,note
712,Brewdog Tokyo,Bière Porter & Stout,Royaume-Uni,Brewdog,33.0,16.5,90.0,100.0,80.0,20.0,80.0,20.0,10.99,4.08
538,Founders KBS,Bière Porter & Stout,Etats-Unis d'Amérique,NaN,36.0,12.3,70.0,100.0,80.0,0.0,80.0,40.0,7.99,4.45
92,Studio Oedipus No. 11,Bière Sour,Pays-Bas,Brasserie Oedipus,38.0,6.5,NaN,NaN,NaN,NaN,NaN,NaN,6.99,3.58
256,Studio Oedipus No. 13,Cidre,Pays-Bas,Brasserie Oedipus,38.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,6.99,3.50
402,Tre Fontane Trappistenbier,Bière Triple,Italie,Abdij Tre Fontane,33.0,8.0,30.0,80.0,60.0,0.0,60.0,0.0,6.99,3.66


Ces valeurs sont cohérentes aussi. La bière la plus chère est également la bière la plus forte. 

### Traitement des valeurs maquantes
On regarde ensuite les valeurs manquantes des bières

In [164]:
df.isnull().sum()

beer_name         1
beer_style        5
country           0
brewery_name    327
bottle            0
abv               0
ibu             413
intensite        55
longueur         55
acidite          55
amertume         55
umami            55
price           147
note             68
dtype: int64

Une bière n'a pas de nom. C'est étrange.

In [166]:
df[df['beer_name'].isnull()]

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,umami,price,note
687,NaN,Bière Triple,Belgique,Brouwerij Wilderen,33.0,8.2,NaN,60.0,60.0,20.0,40.0,0.0,2.79,3.76


Une recherche rapide montre qu'il s'agit de la Triple Kannunik. On remplace cette avleur manquante. 

**NB : on pourrait tout à fait la supprimer mais comme cette donnée est complète, on la garde**

In [168]:
df['beer_name'][687] = 'Triple Kannunik'
# Vérification
#df.iloc[687]

On peut à présent ajouter une colonne égale au prix de la pinte de cette bière, d'après le site www.beerwulf.com. 

Ce sera utile pour comparer les prix par la suite.

In [203]:
df['prix_de_la_pinte'] = 0.0

for i in range(len(df)):
    if df['price'][i] != np.nan:
        df['prix_de_la_pinte'][i] = round(100*50*df['price'][i]/df['bottle'][i])/100
    else:
        df['prix_de_la_pinte'][i] = np.nan

df.head(7)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,umami,price,note,prix_de_la_pinte
0,Brugse Zot blonde,Bière Blonde,Belgique,Brouwerij De Halve Maan,33.0,6.0,23.0,60.0,40.0,20.0,40.0,0.0,2.69,3.71,4.08
1,Brouwerij 't IJ Natte Dubbel,Bière Double,Pays-Bas,Brouwerij 't IJ,33.0,6.5,NaN,60.0,60.0,0.0,40.0,0.0,2.65,3.44,4.02
2,Bruut Gajes Tripel,Bière Triple,Pays-Bas,Bruut Bier,33.0,8.0,38.0,60.0,60.0,40.0,60.0,0.0,2.99,3.76,4.53
3,Baxbier Koud Vuur,Bière Brune,Pays-Bas,Bax Bier Brouwerij,33.0,6.2,38.0,80.0,40.0,20.0,60.0,0.0,3.09,3.37,4.68
4,Bacchus Framboos,Bière Fruitée,Belgique,Brouwerij Van Honsebrouck,38.0,5.0,NaN,80.0,60.0,60.0,20.0,0.0,3.49,3.60,4.59
5,Boon Kriek,Bière Fruitée,Belgique,Brouwerij Boon,38.0,4.0,NaN,60.0,80.0,60.0,20.0,0.0,3.39,3.54,4.46
6,Birra Ichnusa,Lager,Italie,Ichnusa,33.0,4.7,NaN,20.0,40.0,0.0,20.0,0.0,1.79,2.85,2.71
